In [12]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [19]:
from sklearn.model_selection import train_test_split

# Read the dataset
dataset = pd.read_json("../data_without_hashtags.json")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['isSarcastic'], test_size=0.2, random_state=42)

In [20]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from keras_preprocessing.sequence import pad_sequences

# Read the dataset
# dataset = pd.read_json("data_without_hashtags.json")

# Example data
#X_train = ["This is a tweet!", "Another tweet here."]
#X_test = ["Yet another tweet!", "And one more tweet."]

# Define max_length
max_length = 140

# Initialize TweetTokenizer
tweetTokenizer = TweetTokenizer()

# Tokenize training text data
X_train_tokenized = [tweetTokenizer.tokenize(text) for text in X_train]

# Tokenize testing text data
X_test_tokenized = [tweetTokenizer.tokenize(text) for text in X_test]

# Create Tokenizer instance
tokenizer = Tokenizer()

# Fit tokenizer on training text data
tokenizer.fit_on_texts(X_train_tokenized)

# Convert text data to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train_tokenized)
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokenized)

# Pad sequences
X_train = pad_sequences(X_train_sequences, maxlen=max_length)
X_test = pad_sequences(X_test_sequences, maxlen=max_length)

# Display shapes of resulting matrices
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (31824, 140)
Shape of X_test: (7956, 140)


In [21]:
from keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.optimizers import Adam

embedding_dim = 100

# Define the vocabulary size based on the actual number of unique words in the training data
vocab_size = len(tokenizer.word_index) + 1

max_length = 140

optimizer = Adam(learning_rate=0.000009)
m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(CuDNNLSTM(units=150))
m1.add(Dense(units=64))
m1.add(Dense(units=64))
m1.add(Dense(units=1, activation='sigmoid'))

m1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
m1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 140, 100)          3342900   
                                                                 
 cu_dnnlstm_4 (CuDNNLSTM)    (None, 150)               151200    
                                                                 
 dense_6 (Dense)             (None, 64)                9664      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,507,989
Trainable params: 3,507,989
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Train the model
m1.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = m1.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20
498/498 [==============================] - 13s 25ms/step - loss: 0.6903 - accuracy: 0.5348 - val_loss: 0.6878 - val_accuracy: 0.5368
Epoch 2/20
498/498 [==============================] - 12s 24ms/step - loss: 0.6867 - accuracy: 0.5354 - val_loss: 0.6844 - val_accuracy: 0.5377
Epoch 3/20
498/498 [==============================] - 12s 25ms/step - loss: 0.6821 - accuracy: 0.5555 - val_loss: 0.6791 - val_accuracy: 0.5626
Epoch 4/20
498/498 [==============================] - 13s 25ms/step - loss: 0.6733 - accuracy: 0.5967 - val_loss: 0.6682 - val_accuracy: 0.6083
Epoch 5/20
498/498 [==============================] - 12s 25ms/step - loss: 0.6506 - accuracy: 0.6419 - val_loss: 0.6354 - val_accuracy: 0.6471
Epoch 6/20
498/498 [==============================] - 12s 25ms/step - loss: 0.5860 - accuracy: 0.7046 - val_loss: 0.5956 - val_accuracy: 0.6844
Epoch 7/20
498/498 [==============================] - 12s 25ms/step - loss: 0.5330 - accuracy: 0.7418 - val_loss: 0.5553 - val_accuracy:

In [23]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
y_val_pred_prob_m1 = m1.predict(X_test)
y_val_pred_m1 = (y_val_pred_prob_m1 > 0.5).astype(int)  # Threshold for binary classification

# Assuming y_test is in binary format (0 or 1)
y_val_true_m1 = y_test

# Calculate precision and recall for binary classification
precision_m1 = precision_score(y_val_true_m1, y_val_pred_m1)
recall_m1 = recall_score(y_val_true_m1, y_val_pred_m1)

# print the results
print(f'Precision: {precision_m1:.4f}')
print(f'Recall: {recall_m1:.4f}')

249/249 [==============================] - 1s 5ms/step
Precision: 0.7863
Recall: 0.6931
